In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install hazm

     |████████████████████████████████| 317kB 2.8MB/s 
     |████████████████████████████████| 235kB 11.9MB/s 
     |████████████████████████████████| 1.4MB 8.3MB/s 
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp36-cp36m-linux_x86_64.whl size=155148 sha256=26733fa3be7b9e28fb10d94c4d259d299e2f180c45f4e415efd56e357928635a
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
  Created wheel for nltk: filename=nltk-3.3-cp36-none-any.whl size=1394470 sha256=b3e40d0d071a708527db4ce706f5e12318d22526ee5151ae9317019f37929a17
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built libwapiti nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
import collections
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import homogeneity_score

from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy import sparse

import string

import re
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from sklearn.decomposition import LatentDirichletAllocation
from hazm import *

## Data

In [ ]:
def pre_process(text):

    text = ' '.join(x for x in text.split() if x.startswith('http') == False and x.startswith('www') == False)
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'^http?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'^www?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    # extra step to make sure html tags are completely removed
    clean = re.compile('<.*>|<.*\"')
    text = re.sub(clean, '', text)

    # text=re.sub("(\\d|\\W)+"," ",text)
    text = re.sub("(pic.*)","", text)
    text = re.sub(r"http\S+","", text)
    text = re.sub("(twitter.*)","", text)
    text=re.sub("<!--?.*?-->","",text)
    text = re.sub("(dlvr.*)","", text)
    text = text.replace("zoomit", "")
    text=text.replace(u'\u200c', '*')

    return text
 
def pre_process2(text):

    text=re.sub("(\\d|\\W)+"," ",text)
    text = re.sub("(pic.*)","", text)
    text = re.sub(r"http\S+","", text)
    text = re.sub("(twitter.*)","", text)
    text=re.sub("<!--?.*?-->","",text)
    text = re.sub("(dlvr.*)","", text)
    text = text.replace("zoomit", "")
    text=text.replace(u'\u200c', '*')

    return text

def get_stop_words(stop_file_path):

    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

stopwords=get_stop_words("/content/drive/My Drive/Fake news pro/step2.1/stopwords.dat")
newstopwords = ["_ir" ,"bbc" ,"bit" ,"com" ,"dgto" ,"eghtesadnews" ,"fa","http" ,"https","in" ,"ir" ,"ly" ,"news" ,"status" ,"tejaratnews","tiny","twitter","www","استhttp","ها" ,"های","شدhttps","شدhttp"]
initial_stopwords = list(stopwords)

mod_stop_words = initial_stopwords + newstopwords

In [ ]:
data_bbc = pd.read_json("/content/drive/My Drive/Fake news pro/similarity/data/BBCpersian.json",lines=True)
data_bbc = data_bbc.filter(items = ["tweet","created_at"])
data_euro= pd.read_json("/content/drive/My Drive/Fake news pro/similarity/data/Euronews.json",lines=True)
data_euro = data_euro.filter(items = ["tweet","created_at"])

In [ ]:
data_bbc["tweet"] = data_bbc["tweet"].apply(lambda x:pre_process(x))
data_euro["tweet"] = data_euro["tweet"].apply(lambda x:pre_process(x))


In [ ]:
data_bbc

,tweet,created_at
0,کرونا باعث لغو بسیاری از جشن*ها، کنسرت*ها و مه...,2020-08-03 11:40:20
1,بنا بر این داده*ها در بهمن ۲۴۸۲ نفر با علایم م...,2020-08-03 10:59:33
2,این اطلاعات هم*جهت با برآوردهایی است که بی*بی*...,2020-08-03 10:59:33
3,داده*های پزشکی و فهرست*هایی از موارد قطعی و مش...,2020-08-03 10:59:33
4,درز آمار کرونا پنهانکاری یک ماهه ایران را فاش ...,2020-08-03 10:41:08
...,...,...
997,در پی انفجار و آتش سوزی در مرکز درمانی سینا اط...,2020-07-01 08:17:03
998,دانشجوی زنی که کمتر از یک هفته گره ۵۰ ساله ریا...,2020-07-01 07:42:13
999,آمریکا تقریبا تمام موجودی یک داروی کلیدی درمان...,2020-07-01 07:22:00
1000,فرانسه تایید ۵ سال زندان فریبا عادلخواه را محک...,2020-07-01 02:44:43


In [ ]:


list1=[]
time1=[]
for k,i in enumerate( data_bbc["tweet"]):
  a=sent_tokenize(i)
  # print(i)
  for j in a:
    # print(j)
    list1.append(j)
    time1.append(data_bbc["created_at"][k])


In [ ]:
d_bbc= pd.DataFrame()
data_b = pd.DataFrame({"tweet":list1,"time":time1})
d_bbc= d_bbc.append(data_b)
d_bbc["tweet"] = d_bbc["tweet"].apply(lambda x:pre_process2(x))

In [ ]:
list2=[]
time2=[]
for k,i in enumerate(data_euro["tweet"]):
  a=sent_tokenize(i)
  for j in a:
    # print(j)
    list2.append(j)
    time2.append(data_bbc["created_at"][k])


In [ ]:
d_euro= pd.DataFrame()
data_e = pd.DataFrame({"tweet":list2,"time":time2})
d_euro= d_euro.append(data_e)
d_euro["tweet"] = d_euro["tweet"].apply(lambda x:pre_process2(x))


In [ ]:
for i in d_euro["tweet"]:
  mask=d_euro["tweet"]==i
  if len(d_euro["tweet"][mask])>=3:
    mask2=d_euro["tweet"]!=i
    d_euro=d_euro[mask2]
    d_euro = d_euro.reset_index(drop=True)

In [ ]:
for i in d_bbc["tweet"]:
  mask=d_bbc["tweet"]==i
  if len(d_bbc["tweet"][mask])>=3:
    mask2=d_bbc["tweet"]!=i
    d_bbc=d_bbc[mask2]
    d_bbc = d_bbc.reset_index(drop=True)


# TF-IDF

In [ ]:
# cv = TfidfVectorizer(stop_words=mod_stop_words)

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(stop_words=mod_stop_words)
tfidf_transformer = TfidfTransformer()
docs_bbc=d_bbc["tweet"].tolist()
count_tweets_bbc = count_vect.fit_transform(docs_bbc)
total_tweets_bbc=tfidf_transformer.fit_transform(count_tweets_bbc)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آید', 'توان', 'تواند', 'توانند', 'رسد', 'رود', 'سال', 'نمی', 'گوید', 'گویند'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
import datetime
from datetime import timedelta
datetimeFormat = '%Y-%m-%d %H:%M:%S'

def time_finder(min_time,time):
  d_t=datetime.datetime.strptime(str(time), datetimeFormat)- datetime.datetime.strptime(str(min_time), datetimeFormat)
  return abs(d_t.days)

bbc_time=d_bbc["time"].tolist()
bbc_time=np.array(bbc_time)
euro_time=d_euro["time"].tolist()
euro_time=np.array(euro_time)


In [ ]:
from sklearn.metrics.pairwise import linear_kernel
tweet_number_bbc=total_tweets_bbc.shape[0]
label_dic=[]
for i in range(tweet_number_bbc):
  min_time=bbc_time[i]
  mask=[time_finder(min_time,time)<2 for time in bbc_time]
  a=[i for i in range(len(mask)) if mask[i]==True]
  A=total_tweets_bbc
  A=A.tocsr()[a,:]
  
  cosine_similarities = linear_kernel(total_tweets_bbc[i], A).flatten()
  related_docs_indices = cosine_similarities.argsort()[:-3:-1]
  max_similar=np.min(cosine_similarities[related_docs_indices])

  if max_similar > 0.8:
    label_dic.append([a[x] for x in related_docs_indices])


In [ ]:
label_dic=set(map(tuple, label_dic))
label_dic=list(label_dic)
label_dic=np.array(label_dic)

In [ ]:
tweet_label_bbc=np.arange(tweet_number_bbc)
for i in range(tweet_number_bbc):
  mask1=label_dic[:,0]==i
  list1=label_dic[mask1,1]
  if len(list1)!=0:
    for j in list1:
      tweet_label_bbc[j]=tweet_label_bbc[i]


In [ ]:
d_bbc["label"]=tweet_label_bbc
d_bbc.to_json(r"/content/drive/My Drive/Fake news pro/similarity/data/BBCpersian_labeled.json",orient='split')

In [ ]:
docs_euro=d_euro["tweet"].tolist()
count_tweets_euro = count_vect.transform(docs_euro)
total_tweets_euro=tfidf_transformer.transform(count_tweets_euro)

In [ ]:
tweet_number_euro=total_tweets_euro.shape[0]
label_dic=[]
for i in range(tweet_number_euro):
  min_time=euro_time[i]
  mask=[time_finder(min_time,time)<15 for time in bbc_time]
  a=[i for i in range(len(mask)) if mask[i]==True]
  A=total_tweets_bbc
  A=A.tocsr()[a,:]
  cosine_similarities = linear_kernel(total_tweets_euro[i], A).flatten()
  related_docs_indices = cosine_similarities.argsort()[:-2:-1]
  max_similar=np.min(cosine_similarities[related_docs_indices])
  # print(related_docs_indices)
  if max_similar > 0.4:
    label_dic.append([i,a[related_docs_indices[0]]])

In [ ]:
label_dic=np.array(label_dic)

In [ ]:
tweet_label_euro=np.arange(tweet_number_euro)+np.array(2000)
for x in label_dic:
  i=int(x[0])
  j=int(x[1])
  tweet_label_euro[i]=d_bbc["label"][j]

In [ ]:
d_euro["label"]=tweet_label_euro
# d_euro[d_euro["label"]<2000]

In [ ]:
d_euro.to_json(r"/content/drive/My Drive/Fake news pro/similarity/data/Euronews_labeled.json",orient='split')

In [ ]:
# data_euro=pd.read_json("/content/drive/My Drive/Fake news pro/similarity/data/Euronews_labeled.json",orient="split")
# data_bbc=pd.read_json("/content/drive/My Drive/Fake news pro/similarity/data/BBCpersian_labeled.json",orient="split")

In [ ]:
totaldata=d_bbc.append(d_euro,ignore_index=True, sort=False)

In [ ]:
totaldata

,tweet,time,label
0,کرونا باعث لغو بسیاری از جشن ها کنسرت ها و مهم...,2020-08-03 11:40:20,0
1,اما برنامه گذاران هم بیکار ننشسته اند و با وجو...,2020-08-03 11:40:20,1
2,بنا بر این داده ها در بهمن نفر با علایم مربوط ...,2020-08-03 10:59:33,2
3,به نظر می رسد ثبت متمرکز اطلاعات بیماران احتما...,2020-08-03 10:59:33,3
4,این اطلاعات هم جهت با برآوردهایی است که بی بی ...,2020-08-03 10:59:33,4
...,...,...,...
2203,پلیس ایتالیا تن ماده روانگردان آمفتامین تولید ...,2020-07-14 18:31:54,2718
2204,چالش تاریخی مرکل در پی آغاز ریاست دوره ای آلما...,2020-07-14 17:37:17,2719
2205,اولین مخالف دولت محلی هنگ کنگ پس از تصویب قانو...,2020-07-14 17:30:18,1471
2206,جشن خیابانی پایان کرونا در جمهوری چک شهروندان...,2020-07-14 16:48:59,2721


In [ ]:
x=0
label_list=list(set(totaldata["label"])) 
for l in label_list:
  mask=totaldata["label"]==l
  totaldata["label"][mask]=[x]*len(mask[mask==True])
  x+=1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
label_list=list(set(totaldata["label"])) 
for l in label_list:
  mask=totaldata["label"]==l
  if len(mask[mask==True])>1:
    print(totaldata["tweet"][mask])

14      ارتش اسرائیل می گوید چهار نفر را در حال بمب گذ...
1889    انفجار بمب جاده ای در سوریه سه سرباز روس را زخ...
Name: tweet, dtype: object
18      آتش سوزی گسترده کالیفرنیا هزاران نفر را مجبور ...
1493    آتش سوزی جنگلی در کالیفرنیا به هزاران تن از سک...
Name: tweet, dtype: object
23      فضانوردان ناسا بعد از سال با کپسولی آمریکایی ب...
1496      دو فضانورد دراگن از سفر تاریخی به زمین بازگشتند
Name: tweet, dtype: object
30    گفتگویی با ستار رحمانی فعال کارگری بخش دوم
32    گفتگویی با ستار رحمانی فعال کارگری بخش اول
Name: tweet, dtype: object
29    اعتصاب های گسترده کارگران صنعت نفت در ایران کا...
31    اعتصاب های گسترده کارگران صنعت نفت در ایران کا...
Name: tweet, dtype: object
44    کرونا و کنکور در ایران مخالفان برپایی کنکور می...
76    مخالفان برپایی کنکور می گویند که با توجه به شی...
Name: tweet, dtype: object
60    اسناد محرمانه ارتش بریتانیا سوال های تازه ای ر...
67    اسناد محرمانه ارتش بریتانیا سوال های تازه ای ر...
Name: tweet, dtype: object
82         شاخص بورس اوراق به

In [ ]:
totaldata

,tweet,time,label
0,کرونا باعث لغو بسیاری از جشن ها کنسرت ها و مهم...,2020-08-03 11:40:20,0
1,اما برنامه گذاران هم بیکار ننشسته اند و با وجو...,2020-08-03 11:40:20,1
2,بنا بر این داده ها در بهمن نفر با علایم مربوط ...,2020-08-03 10:59:33,2
3,به نظر می رسد ثبت متمرکز اطلاعات بیماران احتما...,2020-08-03 10:59:33,3
4,این اطلاعات هم جهت با برآوردهایی است که بی بی ...,2020-08-03 10:59:33,4
...,...,...,...
2203,پلیس ایتالیا تن ماده روانگردان آمفتامین تولید ...,2020-07-14 18:31:54,1969
2204,چالش تاریخی مرکل در پی آغاز ریاست دوره ای آلما...,2020-07-14 17:37:17,1970
2205,اولین مخالف دولت محلی هنگ کنگ پس از تصویب قانو...,2020-07-14 17:30:18,1415
2206,جشن خیابانی پایان کرونا در جمهوری چک شهروندان...,2020-07-14 16:48:59,1971


In [ ]:
testdata=pd.read_json("/content/drive/My Drive/Fake news pro/similarity/data/tweets (2).json",lines=True)

mask1=testdata["username"]!="bbcpersian"
mask2=testdata["username"]!="euronews_pe"

testdata=testdata[mask1]
testdata=testdata[mask2]
testdata = testdata.reset_index(drop=True)

testdata = testdata.filter(items = ["tweet","created_at"])
testdata["tweet"] =testdata["tweet"].apply(lambda x:pre_process(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [ ]:
list1=[]
time1=[]
for k,i in enumerate( testdata["tweet"]):
  a=sent_tokenize(i)
  # print(i)
  for j in a:
    # print(j)
    list1.append(j)
    time1.append(testdata["created_at"][k])

d_test= pd.DataFrame()
testـdata = pd.DataFrame({"tweet":list1,"time":time1})
d_test= d_test.append(testـdata)
d_test["tweet"] = d_test["tweet"].apply(lambda x:pre_process2(x))
test_time=d_test["time"].tolist()
test_time=np.array(test_time)

In [ ]:
total_time=totaldata["time"].tolist()
total_time=np.array(total_time)

count_vect = CountVectorizer(stop_words=mod_stop_words)
tfidf_transformer = TfidfTransformer()

docs_total=totaldata["tweet"].tolist()
count_tweets_total = count_vect.fit_transform(docs_total)
total_tweets_total=tfidf_transformer.fit_transform(count_tweets_total)

docs_test=d_test["tweet"].tolist()
count_tweets_test = count_vect.transform(docs_test)
total_tweets_test=tfidf_transformer.transform(count_tweets_test)

tweet_number_test=total_tweets_test.shape[0]
label_dic=[]

for i in range(tweet_number_test):
  min_time=test_time[i]
  mask=[time_finder(min_time,time)<15 for time in total_time]
  a=[i for i in range(len(mask)) if mask[i]==True]
  A=total_tweets_total
  A=A.tocsr()[a,:]
  cosine_similarities = linear_kernel(total_tweets_test[i], A).flatten()
  related_docs_indices = cosine_similarities.argsort()[:-2:-1]
  max_similar=np.min(cosine_similarities[related_docs_indices])
  # print(related_docs_indices)
  if max_similar > 0.4:
    label_dic.append([i,a[related_docs_indices[0]]])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آید', 'توان', 'تواند', 'توانند', 'رسد', 'رود', 'سال', 'نمی', 'گوید', 'گویند'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
label_dic=np.array(label_dic)
label_dic
tweet_label_test=np.arange(tweet_number_test)+np.array(4000)
for x in label_dic:
  i=int(x[0])
  j=int(x[1])
  tweet_label_test[i]=totaldata["label"][j]
d_test["label"]=tweet_label_test
d_test[d_test["label"]<4000]

,tweet,time,label
0,کاوشگر میلیارد دلاری ناسا فردا به سمت مریخ پرت...,2020-07-29 17:19:08,245
2,طی هفته های گذشته چین و امارات نیز کاوشگرانی ر...,2020-07-29 17:19:08,754
7,نشریه نیچر می نویسد که ماموریت کاوشگر امید هنج...,2020-07-29 05:56:46,710
9,زنی که پشت ماموریت مریخ امارات قرار دارد ساره...,2020-07-29 05:56:45,754
10,رشته_توییت,2020-07-29 05:56:45,156
...,...,...,...
1599,تیم آژانس فضایی امارات متحده عربی بررسی های و ...,2020-07-09 10:16:08,710
1600,کاوشگر مریخ امارات_متحده_عربی,2020-07-09 10:16:08,754
1601,تیم آژانس فضایی امارات متحده عربی بررسی های و ...,2020-07-09 04:10:25,710
1602,کاوشگر مریخ امارات_متحده_عربی,2020-07-09 04:10:25,754


In [ ]:
mask=d_test["label"]<4000
news_data=d_test[mask]
news_data = news_data.reset_index(drop=True)

ll=news_data["label"].tolist()
res = [] 
for i in ll: 
    if i not in res: 
        res.append(i)


for i in res:
  mask=news_data["label"]==i
  print(i)
  print(news_data["tweet"][mask])
  print(totaldata["tweet"][totaldata["label"]==i])
  print("\n")


245
0      کاوشگر میلیارد دلاری ناسا فردا به سمت مریخ پرت...
322    در کنار امارات جی پی ال ناسا هم خودش رو داره آ...
378    کاوشگر میلیارد دلاری ناسا فردا به سمت مریخ پرت...
700    در کنار امارات جی پی ال ناسا هم خودش رو داره آ...
Name: tweet, dtype: object
256                        کاوشگر ناسا به مریخ فرستاده شد
1563    ماموریت میلیارد دلاری مریخ نورد ناسا برای کشف ...
1578    ناسا مریخ نورد پشتکار را به سمت سیاره سرخ پرتا...
1580     تصاویر زنده از پرتاب راکت حامل مریخ نورد پشتک...
2017    شمارش معکوس برای پرتاب مریخ نورد جدید ناسا نکت...
Name: tweet, dtype: object


754
1      طی هفته های گذشته چین و امارات نیز کاوشگرانی ر...
3       زنی که پشت ماموریت مریخ امارات قرار دارد ساره...
5      ساره امیری بانوی ایرانی الاصل بلوچ ونقش کلیدی ...
8      امارات هم یه کاوشگر فرستاد نه به دور زمین نه ب...
12      از صحرا شروع کردیم و الان به فضا رسیدیم جمله ...
                             ...                        
745    کاوشگر بعد از سه ماه به مریخ میرسه و یکی از چا...
746    کاوشگر امید 

# Gensim Mallet

In [ ]:
%%capture
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

In [ ]:

!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

--2020-08-27 23:17:35--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip.3’

mallet-2.0.8.zip.3  100%[===================>]  15.43M  10.7MB/s    in 1.4s    

2020-08-27 23:17:37 (10.7 MB/s) - ‘mallet-2.0.8.zip.3’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
replace mallet-2.0.8/bin/classifier2info? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: mallet-2.0.8/bin/classifier2info  
replace mallet-2.0.8/bin/csv2classify? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: mallet-2.0.8/bin/csv2classify  
replace mallet-2.0.8/bin/csv2vectors? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: mallet-2.0.8/bin/csv2vectors  
replace mallet-2.0.8/bin/mallet? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  infla

In [ ]:
os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
mallet_path = '/content/mallet-2.0.8/bin/mallet' # you should NOT need to change this 
corpus_path = 'transcripts' # you need to change this path to the directory containing your corpus of .txt files

In [ ]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

def clean_text(text):
   
    tokenized_text=tknzr.tokenize(text)
    cleaned_text = [t for t in tokenized_text if t not in mod_stop_words]
    return cleaned_text
 

tokenized_data_bbc = []
tokenized_data_euro = []

docs_bbc=data_bbc["tweet"].tolist()
docs_euro=data_euro["tweet"].tolist()

for text in docs_bbc:

  tokenized_data_bbc.append(clean_text(text))

for text in docs_euro:

  tokenized_data_euro.append(clean_text(text))
 

dictionary_bbc = corpora.Dictionary(tokenized_data_bbc)
dictionary_euro = corpora.Dictionary(tokenized_data_euro)

corpus_bbc = [dictionary_bbc.doc2bow(text) for text in tokenized_data_bbc]
corpus_euro = [dictionary_euro.doc2bow(text) for text in tokenized_data_euro]

In [ ]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus_bbc, num_topics=900,optimize_interval=10,  id2word=dictionary_bbc)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
print("LDA Model:")
for idx, topic in ldamallet.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

LDA Model:
Topic: 0 
Words: 0.084*"*" + 0.033*"." + 0.031*"،" + 0.022*"ایران" + 0.014*"'" + 0.010*"کرونا" + 0.008*"؛" + 0.007*""" + 0.007*":" + 0.006*"؟"


Topic: 1 
Words: nan*"گوینده" + nan*"گفتگویش" + nan*"اسماعیلی" + nan*"آرزویشان" + nan*"آرزویش" + nan*"۷۷" + nan*"گویندگی" + nan*"شغل" + nan*"قائم" + nan*"بعدا"


Topic: 2 
Words: nan*"گوینده" + nan*"گفتگویش" + nan*"اسماعیلی" + nan*"آرزویشان" + nan*"آرزویش" + nan*"۷۷" + nan*"گویندگی" + nan*"شغل" + nan*"قائم" + nan*"بعدا"


Topic: 3 
Words: nan*"گوینده" + nan*"گفتگویش" + nan*"اسماعیلی" + nan*"آرزویشان" + nan*"آرزویش" + nan*"۷۷" + nan*"گویندگی" + nan*"شغل" + nan*"قائم" + nan*"بعدا"


Topic: 4 
Words: nan*"گوینده" + nan*"گفتگویش" + nan*"اسماعیلی" + nan*"آرزویشان" + nan*"آرزویش" + nan*"۷۷" + nan*"گویندگی" + nan*"شغل" + nan*"قائم" + nan*"بعدا"


Topic: 5 
Words: nan*"گوینده" + nan*"گفتگویش" + nan*"اسماعیلی" + nan*"آرزویشان" + nan*"آرزویش" + nan*"۷۷" + nan*"گویندگی" + nan*"شغل" + nan*"قائم" + nan*"بعدا"


Topic: 6 
Words: nan*"گوینده" + na

/usr/local/lib/python3.6/dist-packages/gensim/models/wrappers/ldamallet.py:442: RuntimeWarning: invalid value encountered in true_divide
  topic = topic / topic.sum()  # normalize to probability dist



Topic: 792 
Words: nan*"گوینده" + nan*"گفتگویش" + nan*"اسماعیلی" + nan*"آرزویشان" + nan*"آرزویش" + nan*"۷۷" + nan*"گویندگی" + nan*"شغل" + nan*"قائم" + nan*"بعدا"


Topic: 793 
Words: nan*"گوینده" + nan*"گفتگویش" + nan*"اسماعیلی" + nan*"آرزویشان" + nan*"آرزویش" + nan*"۷۷" + nan*"گویندگی" + nan*"شغل" + nan*"قائم" + nan*"بعدا"


Topic: 794 
Words: nan*"گوینده" + nan*"گفتگویش" + nan*"اسماعیلی" + nan*"آرزویشان" + nan*"آرزویش" + nan*"۷۷" + nan*"گویندگی" + nan*"شغل" + nan*"قائم" + nan*"بعدا"


Topic: 795 
Words: nan*"گوینده" + nan*"گفتگویش" + nan*"اسماعیلی" + nan*"آرزویشان" + nan*"آرزویش" + nan*"۷۷" + nan*"گویندگی" + nan*"شغل" + nan*"قائم" + nan*"بعدا"


Topic: 796 
Words: nan*"گوینده" + nan*"گفتگویش" + nan*"اسماعیلی" + nan*"آرزویشان" + nan*"آرزویش" + nan*"۷۷" + nan*"گویندگی" + nan*"شغل" + nan*"قائم" + nan*"بعدا"


Topic: 797 
Words: nan*"گوینده" + nan*"گفتگویش" + nan*"اسماعیلی" + nan*"آرزویشان" + nan*"آرزویش" + nan*"۷۷" + nan*"گویندگی" + nan*"شغل" + nan*"قائم" + nan*"بعدا"


Topic: 798 
Wor

In [ ]:

def format_topics_sentences(ldamodel=ldamallet, corpus=corpus_bbc, texts=docs_bbc):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus_bbc, texts=docs_bbc)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)
data_bbc["topics"]=df_dominant_topic["Dominant_Topic"]


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
for i in range(50):
  mask=data_bbc["topics"]==i
  print(i)
  print(data_bbc["tweet"][mask])
  print("\n")

0
47      گفتگویی با روجا اسدی
1245                 بخش دوم
Name: tweet, dtype: object


1
149               گزارش کامل را امشب در شصت دقیقه ببینید
510    اما با پیدایش دانش کشاورزان شروع به کشت انواع ...
699    دختر کلاشینکوف بدست غور به دستور رئیس جمهور به...
872    درگیری در بهبهان فضای امنیتی در بعضی از شهرهای...
Name: tweet, dtype: object


2
Series([], Name: tweet, dtype: object)


3
924     نخستین بار ده سال پیش هنگامی که کامبیز شریف به...
1432    بررسی سخن های بهروز کمالوندی در گفت وگو با پور...
Name: tweet, dtype: object


4
740     نه غزه نه لبنان جانم فدای ایران تکرار این شعار...
943                                برنامه این هفته آپارات
1201                               برنامه این هفته آپارات
Name: tweet, dtype: object


5
Series([], Name: tweet, dtype: object)


6
Series([], Name: tweet, dtype: object)


7
97      این اسناد چه اطلاعات دیگری نشان می دهد
1306                     گزارش ایلگین کارلیداغ
Name: tweet, dtype: object


8
473                              چرا شاملو ش

In [ ]:
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=tokenized_data_bbc, dictionary=dictionary_bbc, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


Coherence Score:  0.641358395345158


# Gensim

In [ ]:
%%capture
!pip install pyLDAvis

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [ ]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

def clean_text(text):
   
    tokenized_text=tknzr.tokenize(text)
    cleaned_text = [t for t in tokenized_text if t not in mod_stop_words]
    return cleaned_text
 

tokenized_data_bbc = []
tokenized_data_euro = []

docs_bbc=data_bbc["tweet"].tolist()
docs_euro=data_euro["tweet"].tolist()

for text in docs_bbc:

  tokenized_data_bbc.append(clean_text(text))

for text in docs_euro:

  tokenized_data_euro.append(clean_text(text))
 

dictionary_bbc = corpora.Dictionary(tokenized_data_bbc)
dictionary_euro = corpora.Dictionary(tokenized_data_euro)

corpus_bbc = [dictionary_bbc.doc2bow(text) for text in tokenized_data_bbc]
corpus_euro = [dictionary_euro.doc2bow(text) for text in tokenized_data_euro]

In [ ]:
lda_model = models.LdaModel(corpus=corpus_bbc,num_topics=800 ,id2word=dictionary_bbc,random_state=50,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto",
                                           per_word_topics=True)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a diffe

In [ ]:
print("LDA Model:")
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

LDA Model:
Topic: 0 
Words: 0.000*"همدانی" + 0.000*"قائم" + 0.000*"آرزویش" + 0.000*"گویندگی" + 0.000*"گوینده" + 0.000*"گفتگویش" + 0.000*"اسماعیلی" + 0.000*"لس" + 0.000*"مارچلو" + 0.000*"آنجلس"


Topic: 1 
Words: 0.375*"دولت" + 0.198*"طالبان" + 0.157*"زندانی" + 0.080*"آزاد" + 0.060*"افغانستان" + 0.028*"کردیم" + 0.018*"عید" + 0.018*"قربان" + 0.000*"گوینده" + 0.000*"آرزویش"


Topic: 2 
Words: 0.000*"همدانی" + 0.000*"قائم" + 0.000*"آرزویش" + 0.000*"گویندگی" + 0.000*"گوینده" + 0.000*"گفتگویش" + 0.000*"اسماعیلی" + 0.000*"لس" + 0.000*"مارچلو" + 0.000*"آنجلس"


Topic: 3 
Words: 0.000*"همدانی" + 0.000*"قائم" + 0.000*"آرزویش" + 0.000*"گویندگی" + 0.000*"گوینده" + 0.000*"گفتگویش" + 0.000*"اسماعیلی" + 0.000*"لس" + 0.000*"مارچلو" + 0.000*"آنجلس"


Topic: 4 
Words: 0.000*"همدانی" + 0.000*"قائم" + 0.000*"آرزویش" + 0.000*"گویندگی" + 0.000*"گوینده" + 0.000*"گفتگویش" + 0.000*"اسماعیلی" + 0.000*"لس" + 0.000*"مارچلو" + 0.000*"آنجلس"


Topic: 5 
Words: 0.334*"ابتلا" + 0.137*"شبانه" + 0.114*"اماکن" + 0.104*"

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_data_bbc, dictionary=dictionary_bbc, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.31103716498568235


In [ ]:

a=lda_model.get_document_topics(corpus_euro)
print(a[0])

[(3, 0.019975243), (24, 0.08165488), (37, 0.07981722), (39, 0.0804892), (52, 0.08556259), (65, 0.3971489), (85, 0.08433061), (91, 0.014405714)]


In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus_euro, texts=data_euro):
    sent_topics_df = pd.DataFrame()

    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            

        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return sent_topics_df

docs_euro=data_euro["tweet"].tolist()
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus_bbc, texts=docs_bbc)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)
data_bbc["label"]=df_dominant_topic["Dominant_Topic"]


In [ ]:
data_bbc.head(20)

,tweet,label
0,کرونا باعث لغو بسیاری از جشن ها کنسرت ها و مهم...,65.0
1,بنا بر این داده ها در بهمن نفر با علایم مربوط ...,40.0
2,این اطلاعات هم جهت با برآوردهایی است که بی بی ...,93.0
3,داده های پزشکی و فهرست هایی از موارد قطعی و مش...,98.0
4,درز آمار کرونا پنهانکاری یک ماهه ایران را فاش ...,93.0
5,اخبار جعلی چطور خبرهای جعلی را تشخیص دهیم گاهی...,28.0
6,وزیر کشور تاجیکستان شارمهد در تاجیکستان دستگیر...,92.0
7,ارتش اسرائیل می گوید چهار نفر را در حال بمب گذ...,81.0
8,وزیر بهداشت ایران پیش بینی مرگ سه رقمی را به ر...,15.0
9,کوسه نهنگ یک زن را در غرب استرالیا زخمی کرد,54.0


In [ ]:
data_euro.head(20)

,tweet,label
0,فعالیت کمدین فرانسوی حامی احمدی نژاد در فیسبوک...,65.0
1,درخت ناسا در مریخ دستگاهی که از جو سیاره سرخ ...,65.0
2,حجاج پرتاب سنگ به نماد شیطان را به سبک کرونا ب...,7.0
3,صف خرید ونزوئلایی ها مقابل فروشگاه ایرانی مگا...,40.0
4,پایان یک سفر تاریخی لحظه فرود کپسول حامل فضان...,93.0
5,لحظه فرود کسپول حامل فضانوردان آمریکایی در آب ...,93.0
6,لحظه جدایی کپسول دراگن از ایستگاه فضایی بین ال...,93.0
7,شاخه خراسان داعش مسئولیت حمله مرگبار به زندان ...,98.0
8,آتش سوزی جنگلی در کالیفرنیا به هزاران تن از سک...,98.0
9,سیل بنیان کن و مرگبار در کره جنوبی,28.0


In [ ]:
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

TypeError: ignored

PreparedData(topic_coordinates=                        x                   y  topics  cluster      Freq
topic                                                                   
3     -0.321134+0.000000j  0.010185+0.000000j       1        1  8.573661
91    -0.286549+0.000000j  0.069782+0.000000j       2        1  5.347219
28    -0.166251+0.000000j  0.052527+0.000000j       3        1  4.779659
52    -0.139381+0.000000j  0.040499+0.000000j       4        1  3.309798
86    -0.166373+0.000000j  0.043673+0.000000j       5        1  3.273692
...                   ...                 ...     ...      ...       ...
51     0.039560+0.000000j -0.000755+0.000000j      96        1  0.071312
48     0.039560+0.000000j -0.000755+0.000000j      97        1  0.071312
47     0.039560+0.000000j -0.000755+0.000000j      98        1  0.071312
44     0.039560+0.000000j -0.000755+0.000000j      99        1  0.071312
49     0.039560+0.000000j -0.000755+0.000000j     100        1  0.071312

[100 rows x 5 colum

In [ ]:
def compute_score_for_n(dictionary, corpus, texts,n_topics):

  model = models.LdaModel(corpus=corpus, num_topics=n_topics, id2word=dictionary,random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        
  coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
  coherence_values=coherencemodel.get_coherence()

  return  coherence_values

In [ ]:
cohernc_score=[]
N_cluster = list(range(1,40))
for n in N_cluster:
  print(n)
  s= compute_score_for_n(dictionary=dictionary, corpus=corpus,texts=tokenized_data,n_topics=n)
  cohernc_score.append(s)

In [ ]:
plt.plot(N_cluster,cohernc_score, c ='orange')

plt.title('gensim LDA')
plt.ylabel('coherence_score')
plt.xlabel('number of clusters')
plt.text(45,0.5,'max score is '+str(round(max(cohernc_score),4))+'\nfor '+str(N_cluster[np.argmax(cohernc_score)])+'clusters')
plt.legend()